In [14]:
import requests
import json
import base64
import io
from PIL import Image

In [15]:
urlMovies = "https://api.imdbapi.dev/search/titles?query=El%20Padrino"
headers = {
    'content-type': 'application/json'
}


In [16]:
response = requests.get(urlMovies, headers=headers)

In [17]:
data = response.json()

In [18]:
titles = data['titles']

In [ ]:
titles[0]

In [ ]:
for title in range(len(titles)):
  id = titles[title]['id']
  titleName = titles[title]['originalTitle']
  year = titles[title]['startYear']

  print(f'id: {id}, pelicula: {titleName}, year: {year}')

In [21]:
urlMovies = "https://api.imdbapi.dev/titles/tt0068646"
headers = {
    'content-type': 'application/json'
}
response = requests.get(urlMovies, headers=headers)
data = response.json()


In [ ]:
data

In [ ]:
print(data['primaryTitle'])
print(data['genres'])
print(data['rating']['aggregateRating'])
print(data['startYear'])
print(data['plot'])
print(data['directors'][0]['displayName'])
print(data['primaryImage']['url'])

In [24]:
def reducir_imagen_base64(base64_string, nuevo_ancho=None, calidad=85):
    """
    Reduce el tamaño de una imagen a partir de su cadena Base64.
    """
    if ',' in base64_string:
        header, encoded_data = base64_string.split(',', 1)
    else:
        encoded_data = base64_string

    image_bytes = base64.b64decode(encoded_data)
    img_buffer = io.BytesIO(image_bytes)
    img = Image.open(img_buffer)

    if nuevo_ancho:
        ancho_original, alto_original = img.size
        # Si la imagen ya es más pequeña que el ancho deseado, no la agrandamos
        if ancho_original > nuevo_ancho:
            ratio = alto_original / ancho_original
            nuevo_alto = int(nuevo_ancho * ratio)
            img = img.resize((nuevo_ancho, nuevo_alto), Image.Resampling.LANCZOS)

    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")

    output_buffer = io.BytesIO()
    img.save(output_buffer, format="JPEG", quality=calidad)

    output_buffer.seek(0)
    new_base64_bytes = base64.b64encode(output_buffer.getvalue())
    new_base64_string = new_base64_bytes.decode('utf-8')

    return new_base64_string

In [ ]:
imgUrl = data['primaryImage']['url']

response = requests.get(imgUrl)
response.raise_for_status()
imageDownloaded = response.content

base64Bytes = base64.b64encode(imageDownloaded)
base64String = base64Bytes.decode('utf-8')

print(base64String)

In [26]:
newImageBase64 = reducir_imagen_base64(base64String, 200, 85)

In [ ]:
newImageBase64

In [28]:
def searchMovieByTitle(title):

  urlMovies = f"https://api.imdbapi.dev/search/titles?query={title}"
  headers = {
      'content-type': 'application/json'
  }
  response = requests.get(urlMovies, headers=headers)
  data = response.json()
  titles = data['titles']

  return titles


In [ ]:
dictMovies = searchMovieByTitle("Alien")
len(dictMovies)

In [ ]:
dictMovies[0]['id']

In [31]:
def SearchMovieByIdAndImg(IDMBId, imageSize):
  movieDict = {}


  urlMovies = f"https://api.imdbapi.dev/titles/{IDMBId}"
  headers = {
      'content-type': 'application/json'
  }
  response = requests.get(urlMovies, headers=headers)
  data = response.json()

  try:
     

    imgUrl = data['primaryImage']['url']

    response = requests.get(imgUrl)
    response.raise_for_status()
    imageDownloaded = response.content

    base64Bytes = base64.b64encode(imageDownloaded)
    base64String = base64Bytes.decode('utf-8')

    if ',' in base64String:
        header, encoded_data = base64String.split(',', 1)
    else:
        encoded_data = base64String

    image_bytes = base64.b64decode(encoded_data)
    img_buffer = io.BytesIO(image_bytes)
    img = Image.open(img_buffer)

    if imageSize:
      ancho_original, alto_original = img.size
      # Si la imagen ya es más pequeña que el ancho deseado, no la agrandamos
    if ancho_original > imageSize:
      ratio = alto_original / ancho_original
      nuevo_alto = int(imageSize * ratio)
      img = img.resize((imageSize, nuevo_alto), Image.Resampling.LANCZOS)
    if img.mode in ("RGBA", "P"):
      img = img.convert("RGB")

    output_buffer = io.BytesIO()
    img.save(output_buffer, format="JPEG", quality=85)

    output_buffer.seek(0)
    new_base64_bytes = base64.b64encode(output_buffer.getvalue())
    new_base64_string = new_base64_bytes.decode('utf-8')
  except:
     new_base64_string = None


  movieDict['Titulo'] = data.get('primaryTitle')
  movieDict['Generos']=data.get('genres')
  movieDict['Rate']=data.get('rating',{}).get('aggregateRating','No Rate')
  movieDict['Year']=data.get('startYear', 'No Year')
  movieDict['Summ']=data.get('plot','No summary')
  try:
    movieDict['Director']=data['directors'][0]['displayName']
  except (KeyError, TypeError):
     movieDict['Director'] = 'No Director'
  movieDict['Img']=new_base64_string

  return movieDict




In [ ]:
SearchMovieByIdAndImg('tt0068646', 200)

In [ ]:
from langchain_community.document_loaders import JSONLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document


keyMovies = ["Lord Of the rings", "StarWars","JhonWick"]
key = ""
documentos = []

for movie in keyMovies:
    dictMovie = searchMovieByTitle(movie)
    for movieId in dictMovie:
        id = movieId['id']
        newDictMovie = SearchMovieByIdAndImg(id,200)
        print(newDictMovie)
        contenido = f"Título: {newDictMovie['Titulo']}. Trama: {newDictMovie['Summ']}"
        metadata = {
            "director": newDictMovie['Director'],
            "anio": newDictMovie['Year'],
            "rate": newDictMovie['Rate'],
            "generos": newDictMovie['Generos']
        }
        doc = Document(page_content=contenido, metadata=metadata)
        documentos.append(doc)

    print(f"Procesando {len(documentos)} películas...")
        
        

In [46]:
import os
os.environ["GOOGLE_API_KEY"] = ""
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")


In [ ]:
if os.path.exists("./chroma_db"):
    import shutil
    shutil.rmtree("./chroma_db")

db = Chroma.from_documents(
    documents=documentos,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("¡Éxito! Base de datos vectorial creada usando Google Gemini Embeddings.")

In [ ]:
import google.generativeai as genai
import os
api_key = ""

if not api_key:
    print("¡Error! No encontré la variable GOOGLE_API_KEY")
else:
    genai.configure(api_key=api_key)
    print("Buscando modelos de embedding disponibles para tu cuenta...")
    
    found = False
    for m in genai.list_models():
        if 'embedContent' in m.supported_generation_methods:
            print(f"- {m.name}")
            found = True
            
    if not found:
        print("No se encontraron modelos. Verifica tu API Key o región.")